In [1]:
import sys
import timeit
start_all = timeit.default_timer()

!{sys.executable} -m pip install PyAthena

from pyathena import connect

conn = connect(s3_staging_dir='s3://[REDACTED]/sagemakerquery',
               region_name='ap-southeast-1')

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 1000
pd.options.mode.chained_assignment = None
#pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [4]:
df_frame = pd.read_csv('s3://da-prd-modeling-sagemaker/preprocess/prodrec-item-frame.csv')
df_frame.set_index('item_id', inplace=True)
df_frame_backup = df_frame.copy()

                deprice zframe_brand_display  creation_timestamp  \
item_id                                                            
1100000000029  490000.0               BALENO          1413306000   
1100000000096  400000.0             BENETTON          1181062800   
1100000000358  400000.0               DIESEL          1256490000   
1100000000511  340000.0               DISNEY          1373562000   
1100000000552  260000.0               DISNEY          1278954000   
...                 ...                  ...                 ...   
121501609456        NaN                 NIKE                   0   
122406171455        NaN          CATERPILLAR                   0   
123802806556        NaN         MERCEDEZBENZ                   0   
124304227452        NaN                   GZ                   0   
134000748252        NaN         PIERRECARDIN                   0   

               zframe_base_curve  zframe_bridge  zframe_diagonal  zframe_size  \
item_id                           

In [5]:
# remove SKU which have 0-XFactor

df_frame['xfactor'] = df_frame['xfactor'].astype(int) 
df_frame = df_frame[df_frame['xfactor'] != 0]
df_frame_backup = df_frame.copy()

                 deprice zframe_brand_display  creation_timestamp  \
item_id                                                             
1100000000583   249600.0               DISNEY          1278954000   
1100000001104   331200.0              MATSUDA          1053882000   
1100000001399   393600.0                 PURE          1177261200   
1100000006308  1395000.0         MERCEDEZBENZ          1539968400   
1100000006408  1045000.0         MERCEDEZBENZ          1539968400   
...                  ...                  ...                 ...   
1100000055072   305000.0                  NaN          1614704400   
1100000055244  1070000.0             CONVERSE          1620752400   
1100000056398   660000.0              HANGTEN          1627318800   
1100000056406   660000.0              HANGTEN          1627318800   
1100000056792  2230000.0                 NIKE          1625590800   

               zframe_base_curve  zframe_bridge  zframe_diagonal  zframe_size  \
item_id              

In [ ]:
# remove the non-included columns

#not_included = ["deprice", "ZFRAME_BRAND_DISPLAY", "CREATION_TIMESTAMP", 'brand_category']
not_included = ["deprice", "zframe_brand_display", "creation_timestamp", 'brand_category', 'xfactor']
#not_included = ["deprice", "chmhierdesc", "ZFRAME_BRAND_DISPLAY", "CREATION_TIMESTAMP"]
#not_included = ["deprice", "chmhierdesc", "CREATION_TIMESTAMP"]
df_frame = df_frame[[a for a in list(df_frame) if a not in not_included]]

In [ ]:
# replace the brand value

df_frame.replace({'chmhierdesc': {
    'D-High Brand':6,
    'High Brand':6,
    'Medium High Brand':5,
    'D-Medium Brand':4,
    'Medium Brand':4,
    'Medium Low Brand':3,
    'D-Low Brand':2,
    'Low Brand':2,
    'House Brand':1,
    'Child Medium':3,
    'Child Low':2,
    'D-Child':1,
    'Reading High':4,
    'Reading Medium':3,
    'Reading Paket':2,
    'D-Reading':1,
    'PC Glasses':1,
    'Murah': 0
    }}, inplace=True)


In [ ]:
# value correction, resulting from wrong scale

#df_frame.loc[df_frame['ZFRAME_BASE_CURVE'] > 10, 'ZFRAME_BASE_CURVE'] = df_frame['ZFRAME_BASE_CURVE']/10
df_frame.loc[df_frame['zframe_base_curve'] > 10, 'zframe_base_curve'] = df_frame['zframe_base_curve']/10
#df_frame.loc[(df_frame['ZFRAME_TEMPLE_LENGTH'] > 0) & (df_frame['ZFRAME_TEMPLE_LENGTH'] < 20), 'ZFRAME_TEMPLE_LENGTH'] = df_frame['ZFRAME_TEMPLE_LENGTH']*10
df_frame.loc[(df_frame['zframe_temple_length'] > 0) & (df_frame['zframe_temple_length'] < 20), 'zframe_temple_length'] = df_frame['zframe_temple_length']*10
#df_frame.loc[(df_frame['ZFRAME_VERTICAL'] > 100), 'ZFRAME_VERTICAL'] = df_frame['ZFRAME_VERTICAL']/10
df_frame.loc[(df_frame['zframe_vertical'] > 100), 'zframe_vertical'] = df_frame['zframe_vertical']/10

In [ ]:
# replacing 0 with mean for those dimensions that 0 will be impossible

#df_frame.loc[df_frame['ZFRAME_BRIDGE'] == 0, 'ZFRAME_BRIDGE'] =\
 #   df_frame[df_frame['ZFRAME_BRIDGE'] != 0]['ZFRAME_BRIDGE'].mean()
df_frame.loc[df_frame['zframe_bridge'] == 0, 'zframe_bridge'] =\
    df_frame[df_frame['zframe_bridge'] != 0]['zframe_bridge'].mean()
#df_frame.loc[df_frame['ZFRAME_DIAGONAL'] == 0, 'ZFRAME_DIAGONAL'] =\
 #   df_frame[df_frame['ZFRAME_DIAGONAL'] != 0]['ZFRAME_DIAGONAL'].mean()
df_frame.loc[df_frame['zframe_diagonal'] == 0, 'zframe_diagonal'] =\
    df_frame[df_frame['zframe_diagonal'] != 0]['zframe_diagonal'].mean()
#df_frame.loc[df_frame['ZFRAME_SIZE'] == 0, 'ZFRAME_SIZE'] =\
 #   df_frame[df_frame['ZFRAME_SIZE'] != 0]['ZFRAME_SIZE'].mean()
df_frame.loc[df_frame['zframe_size'] == 0, 'zframe_size'] =\
    df_frame[df_frame['zframe_size'] != 0]['zframe_size'].mean()
#df_frame.loc[df_frame['ZFRAME_TEMPLE_LENGTH'] == 0, 'ZFRAME_TEMPLE_LENGTH'] =\
    #df_frame[df_frame['ZFRAME_TEMPLE_LENGTH'] != 0]['ZFRAME_TEMPLE_LENGTH'].mean()
df_frame.loc[df_frame['zframe_temple_length'] == 0, 'zframe_temple_length'] =\
    df_frame[df_frame['zframe_temple_length'] != 0]['zframe_temple_length'].mean()
#df_frame.loc[df_frame['ZFRAME_VERTICAL'] == 0, 'ZFRAME_VERTICAL'] =\
    #df_frame[df_frame['ZFRAME_VERTICAL'] != 0]['ZFRAME_VERTICAL'].mean()
df_frame.loc[df_frame['zframe_vertical'] == 0, 'zframe_vertical'] =\
    df_frame[df_frame['zframe_vertical'] != 0]['zframe_vertical'].mean()

In [ ]:
# imputation with mean

#columns = ['ZFRAME_BASE_CURVE', 'ZFRAME_BRIDGE', 'ZFRAME_DIAGONAL', 'ZFRAME_SIZE', 'ZFRAME_TEMPLE_LENGTH', 'ZFRAME_VERTICAL', 'r', 'g', 'b']
columns = ['zframe_base_curve', 'zframe_bridge', 'zframe_diagonal', 'zframe_size', 'zframe_temple_length', 'zframe_vertical', 'r', 'g', 'b']

for c in columns:
    df_frame[c] = df_frame[c].fillna(df_frame[c].mean())

In [ ]:
# ordinal encoding + mean imputation
df_frame.replace({'zframe_face_fit': {
        'NR': 0,
        'MD': 1,
        'WD': 2
}}, inplace=True)
df_frame['zframe_face_fit'].fillna(df_frame['zframe_face_fit'].mean(), inplace=True)

In [ ]:
# map the nylor metal and nylor into one category
df_frame.replace({'zframe_style': {
        'NM': 'NY'
}}, inplace=True)

In [ ]:
# map the ZFRAME_MADE_OF into more generic category
df_frame.replace({'zframe_made_of': {
    'ACE': 'PLA',
    'CAR': 'PLA',
    'CGL': 'PLA',
    'POP': 'PLA',
    'PCI': 'PLA',
    'PUL': 'PLA',
    'TR9': 'PLA',
    'ALU': 'MTL',
    'FLO': 'MTL',
    'STS': 'MTL',
    'TIT': 'MTL',
}}, inplace=True)

In [ ]:
# one hot encoding

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = df_frame[['ZFRAME_PRODUCT_SILHOUETTE', 'ZFRAME_MADE_OF', 'ZFRAME_STYLE', 'category']]
X = df_frame[['zframe_product_silhouette', 'zframe_made_of', 'zframe_style', 'category']]
X.fillna('na', inplace=True)
enc.fit(X)
df_onehot = pd.DataFrame(enc.transform(X).toarray(), index=df_frame.index,
                         #columns=enc.get_feature_names(['ZFRAME_PRODUCT_SILHOUETTE', 'ZFRAME_MADE_OF', 'ZFRAME_STYLE', 'category']))
                         columns=enc.get_feature_names(['zframe_product_silhouette', 'zframe_made_of', 'zframe_style', 'category']))

df_frame = df_frame.merge(df_onehot, left_index=True, right_index=True)
#df_frame.drop(['ZFRAME_FACE_FIT', 'ZFRAME_PRODUCT_SILHOUETTE', 'ZFRAME_MADE_OF', 'ZFRAME_STYLE', 'category', 'chmhierdesc',
#              'ZFRAME_PRODUCT_SILHOUETTE_NA', 'ZFRAME_MADE_OF_NA', 'ZFRAME_STYLE_NA', 'category_NA'], axis=1, inplace=True)
df_frame.drop(['zframe_face_fit', 'zframe_product_silhouette', 'zframe_made_of', 'zframe_style', 'category', 'chmhierdesc',
              'zframe_product_silhouette_na', 'zframe_made_of_na', 'zframe_style_na', 'category_na'], axis=1, inplace=True)

In [ ]:
# normalize using minmax scaler (min 0, max 1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1))
df_scaled = pd.DataFrame(scaler.fit_transform(df_frame), columns=df_frame.columns)

In [ ]:
# give weight
# TODO: to be adjusted later

# df_scaled[['ZFRAME_VERTICAL']] =\
#     10*df_scaled[['ZFRAME_VERTICAL']]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# TODO: The precision of the result needs to be increased
# there are too many features

cos_sim = cosine_similarity(df_scaled)
#cos_sim = cosine_similarity(df_scaled, df_scaled.iloc[0:1,:])
indices = pd.Series(df_frame.index)

In [ ]:
#set number of recommendations to make
n = 25
def recommendations_with_price(name, cos_sim = cos_sim):
    m = 65 #number of data that is included by the first filter (to minimize run time)
        
    top_n_recommended_frame_inclusive = []
    top_n_recommended_frame = []
    top_n_score_series = []

    idx = indices[indices == name].index[0]
    #cos_sim_idx = cosine_similarity(df_scaled, df_scaled.iloc[idx:(idx+1),:])

    # Create series based on the similarity score
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    #score_series = pd.Series(cos_sim_idx.reshape(-1)).sort_values(ascending = False)
    top_n_score_series_inclusive = score_series.iloc[0:(m+1)]
    
    # Get index and create n most similar recommendation
    top_n_indexes_inclusive = list(score_series.iloc[0:(m+1)].index)

    # Create the thresholds for filtering
    recom_index = list(df_frame.index)[idx]
    recom_price = df_frame_backup.loc[recom_index, 'deprice']
    allowed_max = recom_price + 1000000
    min_price = recom_price*0.8
    max_price = recom_price*1.5
    if max_price > allowed_max:
        max_price = allowed_max
       
    # Filter price based on chosen product, put it in list
    for j in top_n_indexes_inclusive:
        #print(j)
        if pd.isnull(recom_price):
            top_n_recommended_frame.append(list(df_frame.index)[j])
            top_n_score_series.append(score_series.loc[j])
        else:
            prod_idx = list(df_frame.index)[j]
            prod_price = df_frame_backup.loc[prod_idx, 'deprice']
            if prod_price > min_price and prod_price < max_price:
                top_n_recommended_frame.append(list(df_frame.index)[j])
                top_n_score_series.append(score_series.loc[j])

    df_recommend = df_frame_backup.loc[top_n_recommended_frame, :]
    df_recommend['score'] = top_n_score_series
    df_recommend = df_recommend.head(n+1)
    df_recommend = df_recommend.index.tolist()
    
    if name in df_recommend:
        df_recommend.insert(0, df_recommend.pop(df_recommend.index(name)))
    else:
        df_recommend.append(name)
        df_recommend.insert(0, df_recommend.pop(df_recommend.index(name)))
    
    return df_recommend

In [ ]:
#determine the number of items to put into list
index = df_frame_backup.index
index = index.to_series()
#index = index.iloc[0:10]

In [ ]:
#running the main function for each dataset
import multiprocessing as mp
import timeit

start = timeit.default_timer()

with mp.Pool(mp.cpu_count()) as pool:
#with mp.Pool(16) as pool:
    frame_lists = pool.map(recommendations_with_price, index)

frame_final = pd.DataFrame(frame_lists)

stop = timeit.default_timer()
print('Time: ', stop - start)

In [ ]:
# create a backup and new variables for checking
frame_final_backup = frame_final.copy()
frame_final = frame_final.set_index(0)
#frame_final = frame_final.drop_duplicates()
final_list = frame_final.index.to_list()
index_list = index.to_list()
final_list_dropped = frame_final.index.to_list()

# validate whether there are any duplicates or not
def find_duplicates(L):
    seen = set()
    seen2 = set()
    seen_add = seen.add
    seen2_add = seen2.add
    for item in L:
        if item in seen:
            seen2_add(item)
        else:
            seen_add(item)
    return list(seen2)

find_duplicates(index_list)

In [ ]:
frame_final.reset_index(inplace=True)
frame_final.rename(columns={0: "item_id"}, inplace=True)
frame_final = frame_final.astype('Int64')
#frame_final.drop(26, axis=1, inplace=True)

In [ ]:
# Save the file
frame_final.to_csv('s3://da-prd-modeling-sagemaker/output/prodrecframe_itemsim.csv', index=False)

stop_all = timeit.default_timer()
print('Time: ', stop_all - start_all)

# Uncomment below to read the file
#frame_final = pd.read_csv('s3://da-prd-modeling-sagemaker/output/prodrecframe-itemsim/prodrecframe-itemsim-final.csv')